In [1]:
!pip install sparqlwrapper

    100% |████████████████████████████████| 235kB 3.6MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 16.9MB/s ta 0:00:01


In [3]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# Get cities
query = """SELECT DISTINCT ?ocean ?oceanLabel ?image {
  ?grandeville wdt:P31 wd:Q1549591;
               wdt:P17 ?pays;
               wdt:P18 ?image.
 SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
LIMIT 10"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append((result["grandevilleLabel"]["value"], result["paysLabel"]["value"], result["image"]["value"]))

QueryBadFormed: QueryBadFormed: a bad request has been sent to the endpoint, probably the sparql query is bad formed. 

Response:
b'SPARQL-QUERY: queryStr=SELECT DISTINCT ? ?grandevilleLabel ?pays ?paysLabel ?image {\n  ?grandeville wdt:P31 wd:Q1549591;\n               wdt:P17 ?pays;\n               wdt:P18 ?image.\n SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }\n}\nLIMIT 10\njava.util.concurrent.ExecutionException: org.openrdf.query.MalformedQueryException: Encountered " "?" "? "" at line 1, column 17.\nWas expecting one of:\n    "(" ...\n    "*" ...\n    <VAR1> ...\n    <VAR2> ...\n    \n\tat java.util.concurrent.FutureTask.report(FutureTask.java:122)\n\tat java.util.concurrent.FutureTask.get(FutureTask.java:206)\n\tat com.bigdata.rdf.sail.webapp.BigdataServlet.submitApiTask(BigdataServlet.java:292)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doSparqlQuery(QueryServlet.java:678)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doGet(QueryServlet.java:290)\n\tat 

In [ ]:
dataframe = pd.DataFrame(array, columns=["ville", "pays", "image"])
dataframe = dataframe.astype(
    dtype={"ville": "<U200", "pays": "<U200", "image": "<U200"}
)
dataframe

In [ ]:
import requests
import shutil
import os

def download_image(url):
    
    headers = {"User-Agent":"Mozilla/5.0"}
    request = requests.get(url,
        allow_redirects=True,
        headers=headers,
        stream=True)
    if request.status_code == 200:
        with open(os.path.basename(url), 'wb') as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

In [ ]:
dataframe.image.apply(download_image)